In [2]:
import os
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
import torch.nn as nn
from torchvision import models
from torch.utils.data import DataLoader
import torch.optim as optim
import time
import copy
from google.colab import drive
drive.mount('/content/drive')

# Путь к папке с изображениями
data_train = '/content/drive/MyDrive/farf_data/data/train/'
data_val = '/content/drive/MyDrive/farf_data/data/val/'
data_test = '/content/drive/MyDrive/farf_data/data/test/'


# Аугментации данных
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(data_train, transform=train_transforms)
val_dataset = datasets.ImageFolder(data_val, transform=val_transforms)
test_dataset = datasets.ImageFolder(data_test, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

Mounted at /content/drive


In [5]:
# Используем предобученную модель EfficientNet
model = models.efficientnet_b0(pretrained=True)

# Заменяем последний слой для нашей задачи
num_ftrs = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(num_ftrs, len(train_dataset.classes))
)

# Используем GPU, если доступно
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


device: cuda:0


In [6]:
# Оптимизатор и scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
criterion = nn.CrossEntropyLoss()

# Функция обучения и валидации
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Каждый epoch имеет этапы обучения и валидации
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Проходим по данным
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # Прямой проход
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Обратный проход и оптимизация только на обучающем этапе
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Глубокое копирование модели
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # Загружаем лучшие веса модели
    model.load_state_dict(best_model_wts)
    return model

In [7]:
%%time
# Создаем словарь с dataloaders
dataloaders = {'train': train_loader, 'val': val_loader}

# Обучаем модель
model = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 2.9809 Acc: 0.2843
val Loss: 2.3901 Acc: 0.3922

Epoch 1/24
----------
train Loss: 2.5122 Acc: 0.3837
val Loss: 1.9833 Acc: 0.5194

Epoch 2/24
----------
train Loss: 2.2006 Acc: 0.4494
val Loss: 1.7555 Acc: 0.5690

Epoch 3/24
----------
train Loss: 2.0293 Acc: 0.5014
val Loss: 1.6278 Acc: 0.6171

Epoch 4/24
----------
train Loss: 1.8772 Acc: 0.5334
val Loss: 1.4295 Acc: 0.6651

Epoch 5/24
----------
train Loss: 1.7893 Acc: 0.5491
val Loss: 1.2651 Acc: 0.6760

Epoch 6/24
----------
train Loss: 1.6242 Acc: 0.5839
val Loss: 1.2598 Acc: 0.6853

Epoch 7/24
----------
train Loss: 1.4216 Acc: 0.6347
val Loss: 1.0767 Acc: 0.7225

Epoch 8/24
----------
train Loss: 1.2985 Acc: 0.6656
val Loss: 1.0254 Acc: 0.7225

Epoch 9/24
----------
train Loss: 1.2304 Acc: 0.6891
val Loss: 1.0175 Acc: 0.7380

Epoch 10/24
----------
train Loss: 1.2002 Acc: 0.6848
val Loss: 1.0067 Acc: 0.7566

Epoch 11/24
----------
train Loss: 1.1820 Acc: 0.6887
val Loss: 0.9795 Acc: 0.7504

Ep

In [13]:
def test_input(model, test_loader):
  model.eval()
  running_loss = 0.0
  running_corrects = 0
  cnt = 0
  # Проходим по данным
  for inputs, labels in test_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)
      loss = criterion(outputs, labels)

      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)
      print(cnt)
      cnt += 1

  epoch_loss = running_loss / len(test_loader.dataset)
  epoch_acc = running_corrects.double() / len(test_loader.dataset)
  print(f"Loss {epoch_loss}, Accuracy {epoch_acc}")
  return (epoch_loss, epoch_acc)

In [14]:
test_input(model, val_loader)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
Loss 0.8035155786889068, Accuracy 0.7922403003754693


(0.8035155786889068, tensor(0.7922, device='cuda:0', dtype=torch.float64))

In [15]:
torch.save(model, 'eff_net_sneakers.pth')